<a href="https://colab.research.google.com/github/m10k1/ml-learn/blob/main/japanese_gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggleの環境変数の設定

[Gemma 2](https://ai.google.dev/gemma?hl=ja)は、Googleが開発した最新(2024年時点)のオープンソースの小規模言語モデル（SLM）です。開発はDeepMindが行っているようです。2024年6月27日にリリースされました。

gemma 2 のモデルはKaggleかhugging faceから利用することが可能です。

[hugging face gemma 2 release](https://huggingface.co/collections/google/gemma-2-release-667d6600fd5220e7b967f315)

[kaggle gemma 2](https://www.kaggle.com/models/google/gemma-2)

今回は、2bのみですが日本語対応版がでているそうなのでそちらを試してみたいと思います。

Gemma 2の概要を知りたい人はYoutubeのGoogle for Developersチャンネルを参照するといいでしょう。

https://www.youtube.com/watch?v=ueACBZDrbTY


* https://huggingface.co/google/gemma-2-2b-jpn-it
* https://www.kaggle.com/models/google/gemma-2-2b-jpn-it

## Kaggleの設定

Kagguleからモデルのデータを取得するのでそのための
環境変数を設定します。

Colabで実行するなら、ユーザー名とパスワードをシークレットに登録しておき、userdataを通じて環境変数を設定するのがいいでしょう。

具体的には以下の通りです。

In [7]:
import os

from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')


実行に必要な必要なライブラリを読み込みます。

* kaggulehub
* immutabledict
* sentencepiece

immutabledictは、その名の通りイミュータブルな辞書オブジェクトを扱うためのライブラリです。

SentencePieceは、主にニューラルネットワークベースのテキスト生成システムのための教師なしテキストトークナイザです。




In [8]:
!pip install kagglehub --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: kagglehub
    Found existing installation: kagglehub 0.3.5
    Uninstalling kagglehub-0.3.5:
      Successfully uninstalled kagglehub-0.3.5


In [9]:
!pip install -q -U immutabledict sentencepiece

## gemma 2の実装のレポジトリをクローン

gitで公開されているgemma2の実装をクローンします。


In [10]:
!git clone https://github.com/google/gemma_pytorch.git

Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 239 (delta 86), reused 53 (delta 53), pack-reused 121 (from 2)
Receiving objects: 100% (239/239), 2.18 MiB | 5.15 MiB/s, done.
Resolving deltas: 100% (135/135), done.


クローンしたファイルからgemmaを/content配下に移動
ライブラリの検索パスに追加します。


In [11]:
!mkdir /content/gemma/
!mv /content/gemma_pytorch/gemma/* /content/gemma/


In [12]:
import sys
import shutil
sys.path.append("/content/gemma_pytorch/")
from gemma.config import GemmaConfig, get_model_config
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
import contextlib
import os
import torch
import kagglehub

## kaggleからパラメータファイルをダウンロード



パラメータファイルをキャッシュからコピー


In [13]:
# Load the model
VARIANT = "2b-v2"
MACHINE_TYPE = "cuda"
model_path = kagglehub.model_download('google/gemma-2-2b-jpn-it/pyTorch/gemma-2-2b-jpn-it')

dest_dir = "/content/gemma/gemma-2-2b-jpn-it/pytorch/gemma-2-2b-jpn-it/1/"

if not os.path.exists(dest_dir):
  os.makedirs(dest_dir)

model_weights_path = os.path.join(dest_dir, "model.ckpt")
shutil.copy(os.path.join(model_path, "model.ckpt"), model_weights_path)

model_tokenizer_path = os.path.join(dest_dir, "tokenizer.model")
shutil.copy(os.path.join(model_path, "tokenizer.model"), model_tokenizer_path)



  0%|          | 0.00/4.89G [00:00<?, ?B/s]



  0%|          | 0.00/4.04M [00:00<?, ?B/s]
  0%|          | 1.00M/4.89G [00:00<32:43, 2.67MB/s]

 25%|██▍       | 1.00M/4.04M [00:00<00:01, 2.70MB/s]
  0%|          | 3.00M/4.89G [00:00<11:51, 7.37MB/s]

100%|██████████| 4.04M/4.04M [00:00<00:00, 8.07MB/s]

  0%|          | 9.00M/4.89G [00:00<04:05, 21.4MB/s]
  0%|          | 15.0M/4.89G [00:00<02:44, 31.9MB/s]
  0%|          | 21.0M/4.89G [00:00<02:11, 39.7MB/s]
  1%|          | 26.0M/4.89G [00:00<02:00, 43.1MB/s]
  1%|          | 33.0M/4.89G [00:01<01:47, 48.3MB/s]
  1%|          | 39.0M/4.89G [00:01<01:52, 46.1MB/s]
  1%|          | 44.0M/4.89G [00:01<01:54, 45.4MB/s]
  1%|          | 51.0M/4.89G [00:01<01:45, 49.1MB/s]
  1%|          | 58.0M/4.89G [00:01<01:40, 51.6MB/s]
  1%|▏         | 65.0M/4.89G [00:01<01:37, 53.3MB/s]
  1%|▏         | 71.0M/4.89G [00:01<01:49, 47.2MB/s]
  2%|▏         | 78.0M/4.89G [00:02<01:43, 50.0MB/s]
  2%|▏         | 83.0M/4.89G [00:02<01:50, 46.5MB/s]
  2%|▏         | 89.0M/4.89G [00:02<01:47, 47.9MB/

'/content/gemma/gemma-2-2b-jpn-it/pytorch/gemma-2-2b-jpn-it/1/tokenizer.model'

In [14]:
@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
  """Sets the default torch dtype to the given dtype."""
  torch.set_default_dtype(dtype)
  yield
  torch.set_default_dtype(torch.float)

model_config = get_model_config(VARIANT)
model_config.tokenizer = model_tokenizer_path

device = torch.device(MACHINE_TYPE)
with _set_default_tensor_type(model_config.get_dtype()):
  model = GemmaForCausalLM(model_config)
  model.load_weights(model_weights_path)
  model = model.to(device).eval()

## プロンプトを実行してみる

実際にプロンプトをモデルに渡して結果を出力してみます。


In [15]:
# Use the model
USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn>\n"

prompt = (
  USER_CHAT_TEMPLATE.format(
    prompt="「おにぎり」をテーマに短編を書いてください"
  )
  + "<start_of_turn>model\n"
)
print(prompt)

result = model.generate(
  prompt,
  device=device,
  output_len=256,
)
print(result)

<start_of_turn>user
「おにぎり」をテーマに短編を書いてください<end_of_turn>
<start_of_turn>model

## おにぎりの物語

白い紙に包まれた、温かい空気の匂い。
「おにぎり」の、穏やかな美しさ。 

その姿は、少年の目を奪う。
米粒の黄金色と、海苔の深い緑。
小さな握り具合は、少年の心を静かに包む。

「これは、僕の大切なもの。」

少年は、そのおにぎりを指で優しく触れた。
初めて作られたのだ。
父親の温かい笑いと、お母さんの優しい言葉...
その言葉が、おにぎりの味に宿った。

「おにぎり」は、ただの食べ物ではなく、
少年の心を映し出す鏡だった。
家族の愛情が詰まった、小さな世界。

少年は、おにぎりを持つと、
その世界は広がる。
自分の気持ちを、
家族の物語を、
おにぎりに包み込んだ。



 
 
 
<end_of_turn>


## gradioを使ってUIを追加




In [16]:
!pip install gradio

In [17]:
import gradio as gr

USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn>\n"


def sendPrompt(prompt, prompt_length):
  prompt = (
    USER_CHAT_TEMPLATE.format(
      prompt=prompt
    )
    + "<start_of_turn>model\n"
  )
  result = model.generate(
    prompt,
    device=device,
    output_len=prompt_length,
  )
  return result



app = gr.Interface(fn=sendPrompt, inputs=["text", "number"], outputs=["text"], allow_flagging='never')

app.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://40a03162e56d70082d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
